In [14]:
from ipywidgets import interact
from numpy.random import normal, seed, uniform
from matplotlib import pyplot as plt
from functools import partial
import torch

seed(42)

def plot_function(f, tx=None, ty=None, title=None, min=-2, max=2, figsize=(6,4)):
    x = torch.linspace(min, max, steps=100)
    fig,ax = plt.subplots(figsize=figsize)
    ax.plot(x,f(x))
    if tx is not None: ax.set_xlabel(tx)
    if ty is not None: ax.set_ylabel(ty)
    if title is not None: ax.set_title(title)
    return ax

def noise(x, scale):
    return normal(scale=scale, size=x.shape)

def add_noise(x, mult, add):
    return x * (1 + noise(x, mult)) + noise(x, add)

def quad(a, b, c, x):
    return a*x**2 + b*x + c

def mk_quad(a, b, c):
    return partial(quad, a, b, c)

f = mk_quad(3, 2, 1)

x = torch.linspace(-2, 2, steps=20)[:,None]
y = add_noise(f(x), 0.3, 1.5)

@interact(a=1.5, b=1.5, c=1.5)
def plot_quad(a, b, c):
    ax = plot_function(mk_quad(a, b, c))
    ax.scatter(x, y)

interactive(children=(FloatSlider(value=1.5, description='a', max=4.5, min=-1.5), FloatSlider(value=1.5, descr…